<a href="https://colab.research.google.com/github/SilasRu/Oeko3/blob/master/CNN_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SilasRu/Oeko3.git

Cloning into 'Oeko3'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (349/349), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 22883 (delta 26), reused 324 (delta 12), pack-reused 22534
Receiving objects: 100% (22883/22883), 1.26 GiB | 6.77 MiB/s, done.
Resolving deltas: 100% (713/713), done.
Checking out files: 100% (20045/20045), done.


**Importing packages and path handling**

In [2]:
import os
import numpy as np
from keras.preprocessing.image import  ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
import keras.models as km
import keras.layers as kl
import matplotlib.pyplot as plt
import keras
import random

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Dropout
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score

Using TensorFlow backend.


In [0]:
import sys
sys.path.append("[Oeko3\\data\\train\\spectrograms]")
path = os.path.join('Oeko3', 'data', 'train', 'spectrograms')
persons = os.listdir(path)
persons[-2], persons[-1] = persons[-1], persons[-2]

sys.path.append("[Oeko3\\data\\test\\spectrograms]")
path_test = os.path.join('Oeko3', 'data', 'test', 'spectrograms')
persons_test = os.listdir(path_test)

sys.path.append("[Oeko3\\data\\test]")
speaker_list = os.path.join('Oeko3', 'data', 'test', 'speaker_list.csv')

sys.path.append('Oeko3/src')
import data_utils



**Converting train and test spectrograms to matrices**

In [0]:
def convert_train(size):
  x_train = []
  y_train = []
  errors = 0
  for i in range(size):
      sample = random.randint(0,5)
      tempdir = os.path.join(path, persons[sample])
      tempfiles = os.listdir(tempdir)
      amount = 0
      while amount < 10:  
          sample_img = random.sample(tempfiles, 1)
          temp_img = load_img(os.path.join(tempdir,sample_img[0]))
          temp_x = img_to_array(temp_img)/255.
          if temp_x.shape== (480,640,3):
            temp_x = temp_x.reshape((1,)+temp_x.shape)
            if i ==0 and amount == 0:  
                x_train = temp_x
            else:
              x_train = np.concatenate((x_train,temp_x),  axis = 0)
            y_train.append(sample)
            amount +=1
          else:
            errors += 1
#             print(str(persons[i])+'__'+str(tempfiles[j]+'   INVALID SHAPE. CAN NOT ADD TO TRAINING SET'))
  return x_train, y_train          

In [0]:
def convert_test(speaker_list, range_start, range_stop):
  x_test = []
  y_test = []
  errors = 0
  tempdir = path_test
  tempfiles = sorted(persons_test)[range_start: range_stop]
  
  for index, img in enumerate(tempfiles):
      temp_img = load_img(os.path.join(tempdir,img))
      temp_x = img_to_array(temp_img)/255.
      temp_x = temp_x.reshape((1,)+temp_x.shape)
      
      if index==0:
          x_test = temp_x
      else:
        x_test = np.concatenate((x_test,temp_x),  axis = 0)     
      converter = data_utils.Utils()
      y_test = converter.create_y_test(speaker_list)[range_start: range_stop]
  
  return x_test, y_test

**CNN build**

In [6]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(128,strides=3, kernel_size=3, activation="relu",
                 input_shape= (480, 640, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(64,strides=3 ,kernel_size=3, activation="sigmoid"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(len(persons), activation="softmax"))

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


**Fitting the model on the training set**

In [0]:
episodes = 20
for i in range(episodes):
  x_train, y_train = convert_train(20)
  y_cat = to_categorical(y_train, num_classes=6)
  model.fit(x_train, y_train, validation_split = 0.2, epochs=20, shuffle=True)

In [0]:
x_test, y_test = convert_test(speaker_list, 2200,2201) #2004 stop

In [0]:
for i in range(2201,2400,10):
  x, y, = convert_test(speaker_list, i, i+10)
  x_test = np.concatenate((x_test, x) , axis=0)
  y_test.extend(y)

In [0]:
np.save('y_test12', y_test)
np.save('x_test12', x_test)

In [49]:
len(x_test)
len(y_test)

201

**Prediction on the test set and confusion matrix**

In [94]:
prediction = model.predict_classes(x_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))

[[ 10   0   0  97]
 [  0   0   0   0]
 [ 57  14   0 215]
 [ 38  17   0  52]]
0.124


In [21]:
x_test.shape

(1001, 480, 640, 3)